### OT AND DEM

In [16]:
import pandas as pd
import geopandas as gpd
import glob
import os
import warnings
from shapely.geometry import Polygon

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Function to calculate IoU based on area
def calculate_IoU(intersection_area, union_area):
    return intersection_area / union_area if union_area != 0 else 0

# Initialize variables
regions = ['herschel', 'peel', 'gydan', 'kolguev']
results_df = pd.DataFrame(columns=['Region', 'Year', 'iou_previous', 'iou_same', 'iou_following'])

# Load common DEM and OT data
common_DEMs = gpd.read_file('Plotting/labels/common_DEM_labels.geojson')
common_OT = gpd.read_file('Plotting/labels/all_OT_labels.geojson')

#print crs
print(common_DEMs.crs)
print(common_OT.crs)

for region in regions:
    # Append all footprints of the region into one GeoDataFrame
    footprint_files = glob.glob(f'Results/{region}/Footprints/footprint_{region}_*.geojson')
    region_footprints = pd.concat([gpd.read_file(file) for file in footprint_files], ignore_index=True)
    study_area = gpd.read_file(f'Results/{region}/footprints/{region}_area.geojson')
    common_labelling_area = gpd.overlay(region_footprints, study_area, how='intersection')

    # Spatial join DEMs and OT with common labelling area
    DEM_common_lab = gpd.sjoin(common_DEMs, common_labelling_area, how="inner", predicate='intersects')
    OT_common_lab = gpd.sjoin(common_OT, common_labelling_area, how="inner", predicate='intersects')
    OT_common_lab.drop(columns=['index_right'], inplace=True)
    DEM_common_lab.drop(columns=['index_right'], inplace=True)

    # Convert year columns to integer
    OT_common_lab['year'] = OT_common_lab['year'].astype(int)
    DEM_common_lab['year_end'] = DEM_common_lab['year_end'].astype(int)
    DEM_years = DEM_common_lab['year_end'].unique()

    for year_end in DEM_years:
        IoU_per_year = {}
        for scenario, offset in [('iou_previous', -1), ('iou_same', 0), ('iou_following', 1)]:
            OT_filtered = OT_common_lab[OT_common_lab['year'] == year_end + offset]
            DEM_filtered = DEM_common_lab[DEM_common_lab['year_end'] == year_end]

            both_OT_DEM = gpd.overlay(OT_filtered, DEM_common_lab[DEM_common_lab['year_end'] == year_end], how="intersection")
            
            if both_OT_DEM.empty:
                IoU_per_year[scenario] = 0
            else:
                intersection_area = both_OT_DEM.geometry.area.sum()
                total_area_ot = OT_filtered.geometry.area.sum()
                total_area_dem = DEM_common_lab[DEM_common_lab['year_end'] == year_end].geometry.area.sum()
                
                union = gpd.overlay(OT_filtered, DEM_filtered, how="union")
                union_area = union.geometry.area.sum()

                IoU_per_year[scenario] = calculate_IoU(intersection_area, union_area)

            
        results_df = pd.concat([results_df, pd.DataFrame([{
            'Region': region,
            'Year': year_end,
            'iou_previous': IoU_per_year['iou_previous'],
            'iou_same': IoU_per_year['iou_same'],
            'iou_following': IoU_per_year['iou_following']
        }])], ignore_index=True)

# Calculate overall means
overall_means = results_df[['iou_previous', 'iou_same', 'iou_following']].apply(lambda col: col[col > 0].mean(), axis=0)
overall_row = pd.DataFrame([{
    'Region': 'Overall',
    'Year': 'All Years',
    'iou_previous': overall_means['iou_previous'],
    'iou_same': overall_means['iou_same'],
    'iou_following': overall_means['iou_following']
}], index=[len(results_df)])

results_df = pd.concat([results_df, overall_row], ignore_index=True)

results_df.set_index(['Region', 'Year'], inplace=True)

print('Average IoU for DEM and OT in Overlapping Areas by Year and Scenario:')
print(results_df)


EPSG:3413
EPSG:3413
Average IoU for DEM and OT in Overlapping Areas by Year and Scenario:
                   iou_previous  iou_same  iou_following
Region   Year                                           
herschel 2017                 0         0       0.453937
         2018                 0  0.468611       0.404290
         2020           0.46093         0       0.000000
peel     2016                 0         0       0.000000
         2021                 0  0.529768       0.025283
gydan    2020                 0         0       0.056295
         2021                 0  0.069559       0.000000
kolguev  2017                 0         0       0.143108
Overall  All Years      0.46093  0.355979       0.216582


In [17]:
import pandas as pd
import geopandas as gpd
import glob
import os
import warnings
from shapely.geometry import Polygon

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Function to calculate IoU based on area
def calculate_IoU(intersection_area, union_area):
    return intersection_area / union_area if union_area != 0 else 0

# Initialize variables
regions = ['herschel', 'peel', 'gydan', 'kolguev']
results_df = pd.DataFrame(columns=['Region', 'Year', 'iou_previous', 'iou_same', 'iou_following'])

# Load common DEM and OT data
common_DEMs = gpd.read_file('Plotting/labels/common_DEM_labels.geojson')
common_OT = gpd.read_file('Plotting/labels/all_OT_labels.geojson')

for region in regions:
    # Append all footprints of the region into one GeoDataFrame
    footprint_files = glob.glob(f'Results/{region}/Footprints/footprint_{region}_*.geojson')
    region_footprints = pd.concat([gpd.read_file(file) for file in footprint_files], ignore_index=True)
    study_area = gpd.read_file(f'Results/{region}/footprints/{region}_area.geojson')
    common_labelling_area = gpd.overlay(region_footprints, study_area, how='intersection')

    # Spatial join DEMs and OT with common labelling area
    DEM_common_lab = gpd.sjoin(common_DEMs, common_labelling_area, how="inner", predicate='intersects')
    OT_common_lab = gpd.sjoin(common_OT, common_labelling_area, how="inner", predicate='intersects')
    OT_common_lab.drop(columns=['index_right'], inplace=True)
    DEM_common_lab.drop(columns=['index_right'], inplace=True)

    # Convert year columns to integer
    OT_common_lab['year'] = OT_common_lab['year'].astype(int)
    DEM_common_lab['year_end'] = DEM_common_lab['year_end'].astype(int)
    DEM_years = DEM_common_lab['year_end'].unique()

    for year_end in DEM_years:
        IoU_per_year = {}
        for scenario, offset in [('iou_previous', -1), ('iou_same', 0), ('iou_following', 1)]:
            OT_filtered1 = OT_common_lab[OT_common_lab['year'] == year_end + offset]
            DEM_filtered1 = DEM_common_lab[DEM_common_lab['year_end'] == year_end]

            if OT_filtered1.empty or DEM_filtered1.empty:
                IoU_per_year[scenario] = 0
                continue

            # Calculate intersection
            both_OT_DEM = gpd.overlay(OT_filtered1, DEM_filtered1, how="intersection")

            # Filter OT and DEM labels that intersect
            OT_filtered2 = OT_filtered1[OT_filtered1.intersects(DEM_filtered1.unary_union)]
            DEM_filtered2 = DEM_filtered1[DEM_filtered1.intersects(OT_filtered1.unary_union)]

            # Calculate Union
            union = gpd.overlay(OT_filtered2, DEM_filtered2, how="union")


            if both_OT_DEM.empty:
                IoU_per_year[scenario] = 0
            else:
                intersection_area = both_OT_DEM.geometry.area.sum()
                union_area = union.geometry.area.sum()

                IoU_per_year[scenario] = calculate_IoU(intersection_area, union_area)


            
        results_df = pd.concat([results_df, pd.DataFrame([{
            'Region': region,
            'Year': year_end,
            'iou_previous': IoU_per_year['iou_previous'],
            'iou_same': IoU_per_year['iou_same'],
            'iou_following': IoU_per_year['iou_following']
        }])], ignore_index=True)

# Calculate overall means
overall_means = results_df[['iou_previous', 'iou_same', 'iou_following']].apply(lambda col: col[col > 0].mean(), axis=0)
overall_row = pd.DataFrame([{
    'Region': 'Overall',
    'Year': 'All Years',
    'iou_previous': overall_means['iou_previous'],
    'iou_same': overall_means['iou_same'],
    'iou_following': overall_means['iou_following']
}], index=[len(results_df)])

results_df = pd.concat([results_df, overall_row], ignore_index=True)

results_df.set_index(['Region', 'Year'], inplace=True)

print('Average IoU for DEM and OT in Overlapping Areas by Year and Scenario:')
print(results_df)


Average IoU for DEM and OT in Overlapping Areas by Year and Scenario:
                   iou_previous  iou_same  iou_following
Region   Year                                           
herschel 2017                 0         0       0.585231
         2018                 0  0.571943       0.565402
         2020          0.630481         0       0.000000
peel     2016                 0         0       0.000000
         2021                 0  0.620923       0.440571
gydan    2020                 0         0       0.283908
         2021                 0  0.167541       0.000000
kolguev  2017                 0         0       0.310900
Overall  All Years     0.630481  0.453469       0.437202


### OP AND DEM

In [20]:
import pandas as pd
import geopandas as gpd
import glob
import os
import warnings
from shapely.geometry import Polygon

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Function to calculate IoU based on area
def calculate_IoU(intersection_area, union_area):
    return intersection_area / union_area if union_area != 0 else 0

# Initialize variables
regions = ['herschel', 'peel', 'gydan', 'kolguev']
results_df = pd.DataFrame(columns=['Region', 'Year', 'iou_previous', 'iou_same', 'iou_following'])

# Load common DEM and OT data
all_DEMs = gpd.read_file('Plotting\\labels\\all_DEM_labels.geojson')
all_OP = gpd.read_file('Plotting\\labels\\all_OP_labels.geojson')


for region in regions:
    # Filter data for the current region
    DEM_common_lab = all_DEMs[all_DEMs['region'] == region]
    OP_common_lab = all_OP[all_OP['region'] == region]

    # Convert year columns to integer
    OP_common_lab['year'] = OP_common_lab['year'].astype(int)
    DEM_common_lab['year_end'] = DEM_common_lab['year_end'].astype(int)
    DEM_years = DEM_common_lab['year_end'].unique()

    for year_end in DEM_years:
        IoU_per_year = {}
        for scenario, offset in [('iou_previous', -1), ('iou_same', 0), ('iou_following', 1)]:
            OP_filtered = OP_common_lab[OP_common_lab['year'] == year_end + offset]
            DEM_filtered = DEM_common_lab[DEM_common_lab['year_end'] == year_end]

            both_OP_DEM = gpd.overlay(OP_filtered, DEM_filtered, how="intersection")
            
            if both_OP_DEM.empty:
                IoU_per_year[scenario] = 0
            else:
                intersection_area = both_OP_DEM.geometry.area.sum()
                total_area_op = OP_filtered.geometry.area.sum()
                total_area_dem = DEM_filtered.geometry.area.sum()
                
                union = gpd.overlay(OP_filtered, DEM_filtered, how="union")
                union_area = union.geometry.area.sum()

                IoU_per_year[scenario] = calculate_IoU(intersection_area, union_area)

            
        results_df = pd.concat([results_df, pd.DataFrame([{
            'Region': region,
            'Year': year_end,
            'iou_previous': IoU_per_year['iou_previous'],
            'iou_same': IoU_per_year['iou_same'],
            'iou_following': IoU_per_year['iou_following']
        }])], ignore_index=True)

# Calculate overall means
overall_means = results_df[['iou_previous', 'iou_same', 'iou_following']].apply(lambda col: col[col > 0].mean(), axis=0)
overall_row = pd.DataFrame([{
    'Region': 'Overall',
    'Year': 'All Years',
    'iou_previous': overall_means['iou_previous'],
    'iou_same': overall_means['iou_same'],
    'iou_following': overall_means['iou_following']
}], index=[len(results_df)])

results_df = pd.concat([results_df, overall_row], ignore_index=True)

results_df.set_index(['Region', 'Year'], inplace=True)

print('Average IoU for DEM and OP in Overlapping Areas by Year and Scenario:')
print(results_df)


c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

Average IoU for DEM and OP in Overlapping Areas by Year and Scenario:
                   iou_previous  iou_same  iou_following
Region   Year                                           
herschel 2017                 0         0       0.325268
         2018                 0  0.321618       0.289294
         2020          0.229996         0       0.295705
peel     2016                 0         0       0.000000
         2021          0.452912  0.219167       0.151927
gydan    2020          0.143936  0.112061       0.205453
         2021          0.028485  0.023365       0.009336
kolguev  2017                 0         0       0.312476
Overall  All Years     0.213832  0.169053       0.227066


c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [22]:
import pandas as pd
import geopandas as gpd
import glob
import os
import warnings
from shapely.geometry import Polygon

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Function to calculate IoU based on area
def calculate_IoU(intersection_area, union_area):
    return intersection_area / union_area if union_area != 0 else 0

# Initialize variables
regions = ['herschel', 'peel', 'gydan', 'kolguev']
results_df = pd.DataFrame(columns=['Region', 'Year', 'iou_previous', 'iou_same', 'iou_following'])

# Load common DEM and OP data
all_DEMs = gpd.read_file('Plotting/labels/all_DEM_labels.geojson')
all_OP = gpd.read_file('Plotting/labels/all_OP_labels.geojson')

for region in regions:
    # Filter data for the current region
    DEM_common_lab = all_DEMs[all_DEMs['region'] == region]
    OP_common_lab = all_OP[all_OP['region'] == region]

    # Convert year columns to integer
    OP_common_lab['year'] = OP_common_lab['year'].astype(int)
    DEM_common_lab['year_end'] = DEM_common_lab['year_end'].astype(int)
    DEM_years = DEM_common_lab['year_end'].unique()

    for year_end in DEM_years:
        IoU_per_year = {}
        for scenario, offset in [('iou_previous', -1), ('iou_same', 0), ('iou_following', 1)]:
            OP_filtered1 = OP_common_lab[OP_common_lab['year'] == year_end + offset]
            DEM_filtered1 = DEM_common_lab[DEM_common_lab['year_end'] == year_end]

            if OP_filtered1.empty or DEM_filtered1.empty:
                IoU_per_year[scenario] = 0
                continue

            # Calculate intersection
            both_OP_DEM = gpd.overlay(OP_filtered1, DEM_filtered1, how="intersection")

            # Filter OP and DEM labels that intersect
            OP_filtered2 = OP_filtered1[OP_filtered1.intersects(DEM_filtered1.unary_union)]
            DEM_filtered2 = DEM_filtered1[DEM_filtered1.intersects(OP_filtered1.unary_union)]

            # Calculate Union
            union = gpd.overlay(OP_filtered2, DEM_filtered2, how="union")

            if both_OP_DEM.empty:
                IoU_per_year[scenario] = 0
            else:
                intersection_area = both_OP_DEM.geometry.area.sum()
                union_area = union.geometry.area.sum()

                IoU_per_year[scenario] = calculate_IoU(intersection_area, union_area)

        results_df = pd.concat([results_df, pd.DataFrame([{
            'Region': region,
            'Year': year_end,
            'iou_previous': IoU_per_year['iou_previous'],
            'iou_same': IoU_per_year['iou_same'],
            'iou_following': IoU_per_year['iou_following']
        }])], ignore_index=True)

# Calculate overall means
overall_means = results_df[['iou_previous', 'iou_same', 'iou_following']].apply(lambda col: col[col > 0].mean(), axis=0)
overall_row = pd.DataFrame([{
    'Region': 'Overall',
    'Year': 'All Years',
    'iou_previous': overall_means['iou_previous'],
    'iou_same': overall_means['iou_same'],
    'iou_following': overall_means['iou_following']
}], index=[len(results_df)])

results_df = pd.concat([results_df, overall_row], ignore_index=True)

results_df.set_index(['Region', 'Year'], inplace=True)

print('Average IoU for DEM and OP in Overlapping Areas by Year and Scenario:')
print(results_df)


c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

Average IoU for DEM and OP in Overlapping Areas by Year and Scenario:
                   iou_previous  iou_same  iou_following
Region   Year                                           
herschel 2017                 0         0       0.470876
         2018                 0  0.423911       0.424107
         2020          0.417861         0       0.469698
peel     2016                 0         0       0.000000
         2021          0.514313  0.330197       0.242872
gydan    2020          0.253529  0.208129       0.343117
         2021          0.177903  0.236967       0.075915
kolguev  2017                 0         0       0.431951
Overall  All Years     0.340902  0.299801       0.351219


c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Noe\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
